In [1]:
!pip install pandas numpy gensim scikit-learn tensorflow keras

  Obtaining dependency information for gensim from https://files.pythonhosted.org/packages/63/46/5feab9c524a380bfa9f9f1c0d065743280dca30b216ab4c7a231f22dbed7/gensim-4.3.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for smart-open>=1.8.1 from https://files.pythonhosted.org/packages/fc/d9/d97f1db64b09278aba64e8c81b5d322d436132df5741c518f3823824fae0/smart_open-6.4.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 45.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
import gensim
from sklearn.model_selection import train_test_split, GridSearchCV,  ParameterGrid
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from multiprocessing import Pool

# download the data from the given URLs and store them in dataframes
training_data_df = pd.read_csv('../1_DATA/1_0_PROJECT_DATA/training_data.csv')
unlabelled_test_data_df = pd.read_csv('../1_DATA/1_0_PROJECT_DATA/unlabelled_test_data.csv')


# Assuming your DataFrame has columns 'text' for the text and 'difficulty' for the labels
texts = training_data_df['sentence']
labels = training_data_df['difficulty']


# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # Adjust max_features as needed
X = tfidf_vectorizer.fit_transform(texts).toarray()

In [5]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)
y_categorical = to_categorical(y_encoded)

In [6]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Select a random subset of 1000 samples for hyperparameter tuning
subset_indices = np.random.choice(X_train.shape[0], 1000, replace=False)
X_train_subset = X_train[subset_indices]
y_train_subset = y_train[subset_indices]

# Define the model and parameters for grid search
model = GradientBoostingClassifier()
parameters = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}


# Define a function to train a model with a given set of hyperparameters
def train_model(params):
    model = GradientBoostingClassifier(**params)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    return score, params

# Define your hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    # Add other parameters here
}

# Create a list of all parameter combinations
param_list = list(ParameterGrid(param_grid))

# Use multiprocessing to train models in parallel
with Pool(processes=4) as pool:  # Adjust the number of processes based on your CPU
    results = pool.map(train_model, param_list)

# Find the best performing set of parameters
best_score, best_params = max(results, key=lambda x: x[0])

print(f"Best Score: {best_score}")
print(f"Best Parameters: {best_params}")


Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'train_model' on <module '__main__' (built-in)>
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/op

KeyboardInterrupt: 

In [7]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

gbdt_model = GradientBoostingClassifier()
gbdt_model.fit(X_train, y_train)
y_pred_gbdt = gbdt_model.predict(X_test)

# Evaluate GBDT Model
print("GBDT Model Accuracy:", accuracy_score(y_test, y_pred_gbdt))
print(classification_report(y_test, y_pred_gbdt))

# Calculate precision, recall, F1-score, and accuracy
gbdt_precision = precision_score(y_test, y_pred_gbdt, average='weighted')
gbdt_recall = recall_score(y_test, y_pred_gbdt, average='weighted')
gbdt_f1_score = f1_score(y_test, y_pred_gbdt, average='weighted')
gbdt_accuracy = accuracy_score(y_test, y_pred_gbdt)

# Print the performance metrics
print("-------- GBDT Model Performance --------")
print(f"GBDT Model Precision: {gbdt_precision}")
print(f"GBDT Model Recall: {gbdt_recall}")
print(f"GBDT Model F1-Score: {gbdt_f1_score}")
print(f"GBDT Model Accuracy: {gbdt_accuracy}")

KeyboardInterrupt: 

In [33]:

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(np.unique(y_encoded)), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_categorical[np.where(y_encoded == y_train)[0]], epochs=10, batch_size=32, validation_split=0.1)

y_pred_dnn = model.predict(X_test)

# Evaluate DNN Model
print("DNN Model Accuracy:", accuracy_score(y_test, np.argmax(y_pred_dnn, axis=1)))
print(classification_report(y_test, np.argmax(y_pred_dnn, axis=1)))

ValueError: operands could not be broadcast together with shapes (4800,) (3840,) 

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# download the data from the given URLs and store them in dataframes
training_data_df = pd.read_csv('../1_DATA/1_0_PROJECT_DATA/training_data.csv')
unlabelled_test_data_df = pd.read_csv('../1_DATA/1_0_PROJECT_DATA/unlabelled_test_data.csv')

# Assuming your DataFrame has columns 'sentence' for the text and 'difficulty' for the labels
texts = training_data_df['sentence']
labels = training_data_df['difficulty']

# TF-IDF Vectorization with bi-grams
tfidf_vectorizer = TfidfVectorizer(max_features=15000, ngram_range=(1, 2))
X = tfidf_vectorizer.fit_transform(texts).toarray()

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Convert y_train to categorical for DNN
y_train_categorical = to_categorical(y_train, num_classes=len(np.unique(y_encoded)))

# Gradient Boosting Classifier with more estimators
gbdt_model = GradientBoostingClassifier(n_estimators=200)
gbdt_model.fit(X_train, y_train)
y_pred_gbdt = gbdt_model.predict(X_test)

# Evaluate GBDT Model
print("GBDT Model Accuracy:", accuracy_score(y_test, y_pred_gbdt))
print(classification_report(y_test, y_pred_gbdt))

# Deep Neural Network Model with adjusted layers
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(np.unique(y_encoded)), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

model.fit(X_train, y_train_categorical, epochs=20, batch_size=64, validation_split=0.1, callbacks=[early_stopping])

# Predictions
y_pred_dnn = model.predict(X_test)
y_pred_dnn_classes = np.argmax(y_pred_dnn, axis=1)

# Evaluate DNN Model
print("DNN Model Accuracy:", accuracy_score(y_test, y_pred_dnn_classes))
print(classification_report(y_test, y_pred_dnn_classes))

KeyboardInterrupt: 